In [96]:
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import pandas as pd

def clean_potato_position(df):
    """ Cleaning the potato position data. Replace the null records with zeros,
        forward fill missing prices """
    potato_positions.replace('[NULL]', 0, inplace=True)
    potato_positions.sort_values(by=['Product ID', 'Date'], inplace=True)
    potato_positions.fillna(method='ffill', inplace=True)
    potato_positions.sort_values(by='Date', inplace=True)
    return potato_positions


def calculate_customer_stockpile_value(df):
    """ Calculates the total value of the last known customer positions, based on a position
     record for the last trade date. If there is no position record from the last trade date,
     it is assumed that this means they have closed out of all positions."""
    most_recent_date = df['Date'].max()
    df_filtered = df[df.Date == most_recent_date]
    df_grouped = df_filtered.groupby(['Client ID','Last Name'], as_index=False)[['Total Position Value']].sum().sort_values(
        by=["Total Position Value"], ascending=False)
    df_grouped.columns = ['Client ID', 'Last Name', 'Stockpile Value']
    return df_grouped

def calculate_client_total_activity(df):
    """ I will use a change in positions as a sign of activity as I do not have trade
        data. Any quantity change in a product will be assumed to be 1 trade,
        although in reality it could be several."""
    df = df.sort_values(by=['Client ID', 'Product ID', 'Date'], ascending=[True, True, True])
    df['NewTrade'] = df['Quantity'] != df['Quantity'].shift()
    df_grouped = df.groupby(by=['Client ID', 'Last Name'], as_index=False)[['NewTrade']].sum().sort_values(
        by=['NewTrade'], ascending=False)
    df_grouped.columns = ['Client ID', 'Last Name', 'Position Changes']
    return df_grouped

def calculate_product_total_activity(df):
    df = df.sort_values(by=['Client ID', 'Product ID', 'Date'], ascending=[True, True, True])
    df['NewTrade'] = df['Quantity'] != df['Quantity'].shift()
    df_grouped = df.groupby(by=['Product ID', 'Variety Name'], as_index=False)[['NewTrade']].sum().sort_values(
        by=['NewTrade'], ascending=False)
    df_grouped.columns = ['Product ID', 'Variety', 'Position Changes']
    return df_grouped

def calculate_client_monthly_activity(df):
    """ Returns Monthly """
    df = df.sort_values(by=['Client ID', 'Product ID', 'Date'], ascending=[True, True, True])
    df['NewTrade'] = df['Quantity'] != df['Quantity'].shift()
    df_grouped = df.groupby([pd.Grouper(key='Date', freq='M'), 'Client ID', 'Last Name'])[['NewTrade']].sum().sort_values(
        by=['Date', 'NewTrade'], ascending=[True, False])
    df_grouped.columns = ['Number Of Trades']
    return df_grouped

def calculate_product_monthly_prices(df):
    """Returns DataFrame of monthly prices for each product """
    df_grouped = df[['Date', 'Product ID', 'Price']].groupby([pd.Grouper(key='Date', freq='M'), 'Product ID'])
    df_ordered = pd.concat([df_grouped.head(1)]).sort_values(['Product ID', 'Date'])
    return df_ordered

In [97]:
    potato_info = pd.read_excel('2_Data_Files.xlsx', sheet_name='Potatoes_Info')
    client_info = pd.read_excel('2_Data_Files.xlsx', sheet_name='Client_Info')
    potato_positions = pd.read_excel('2_Data_Files.xlsx', sheet_name='Potatoes_Positions')
    potato_positions = clean_potato_position(potato_positions)
    df_joined = pd.merge(pd.merge(client_info, potato_positions, on='Client ID'), potato_info, on='Product ID')
    df_joined['Total Position Value'] = df_joined['Price'] * df_joined['Quantity']
    graph_titlefont = dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )

In [98]:
    # Display Customer Stockpile Value as table
    result = calculate_customer_stockpile_value(df_joined)
    table_result = result.copy(deep=True) #Not efficient, would do this differently 
    table_result['Stockpile Value'] = table_result['Stockpile Value'].map("£{:,.2f}".format)
    table = ff.create_table(table_result)
    py.iplot(table, filename='Customer StockPile Value Table')

In [99]:
    # Display Customer Stockpile Value as barchart
    data = [go.Bar(
        x=result['Last Name'],
        y=result['Stockpile Value'], 
        name='Customer Stockpile Value'
    )]
    layout = go.Layout(
        title='Customer Stockpile Value',
        xaxis=dict(
            title='Customer',
            titlefont=graph_titlefont
    ),
        yaxis=dict(
            title='Stockpile Value',
            titlefont=graph_titlefont
    )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

In [100]:
    # Customer total activity table
    result = calculate_client_total_activity(df_joined).head(15)
    result['Position Changes'] = result['Position Changes'].map('{:,.0f}'.format)
    table = ff.create_table(result)
    py.iplot(table, filename='Total Customer Activity')

In [101]:
    # Display Customer activity over all time as barchart
    data = [go.Bar(
        x=result['Last Name'],
        y=result['Position Changes'], 
        name='Client Activity'
    )]
    layout = go.Layout(
        title='Client Activity',
        xaxis=dict(
            title='Client',
            titlefont=graph_titlefont
    ),
        yaxis=dict(
            title='Number of Position Changes',
            titlefont=graph_titlefont
    )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

In [102]:
    # Potato Activity
    result = calculate_product_total_activity(df_joined).head(25)
    result['Position Changes'] = result['Position Changes'].map('{:,.0f}'.format)
    table = ff.create_table(result)
    py.iplot(table, filename='Total Customer Activity')

In [103]:
calculate_client_monthly_activity(df_joined)

Number Of Trades
Date       Client ID Last Name                    
2020-05-31 38        Caruso                 1031.0
           74        Southerland             243.0
           94        Burchfield              131.0
           50        Kyle                    101.0
           44        Kong                     78.0
           17        Vue                      64.0
           27        Aldridge                 63.0
           64        Giles                    61.0
           30        Culver                   31.0
           89        Lassiter                 25.0
           52        Albright                 14.0
           82        Cheney                    6.0
           21        Woody                     3.0
           78        Proctor                   3.0
           24        Gustafson                 1.0
           49        Browne                    1.0
           59        Mcdermott                 1.0
           18        Hollins                   0.0
2020-06-30 38        Caruso                 1434.0
           74        Southerland             267.0
           94        Burchfield              165.0
           50        Kyle                    113.0
           27        Aldridge                 55.0
           44        Kong                     49.0
           17        Vue                      46.0
           64        Giles                    42.0
           30        Culver                   41.0
           52        Albright                 18.0
           89        Lassiter                 18.0
           59        Mcdermott                 2.0
...                                            ...
2020-07-31 78        Proctor                   9.0
           65        Jeffrey                   6.0
           49        Browne                    5.0
           77        Spivey                    4.0
           52        Albright                  3.0
           82        Cheney                    3.0
           59        Mcdermott                 2.0
           21        Woody                     1.0
           18        Hollins                   0.0
2020-08-31 38        Caruso                 1577.0
           74        Southerland             320.0
           94        Burchfield              163.0
           44        Kong                     91.0
           50        Kyle                     80.0
           27        Aldridge                 77.0
           17        Vue                      64.0
           30        Culver                   27.0
           64        Giles                    25.0
           65        Jeffrey                  14.0
           89        Lassiter                 11.0
           45        Steward                   8.0
           77        Spivey                    8.0
           59        Mcdermott                 4.0
           82        Cheney                    4.0
           52        Albright                  3.0
           21        Woody                     2.0
           78        Proctor                   2.0
           33        Moya                      1.0
           41        Peoples                   1.0
           49        Browne                    1.0

[78 rows x 1 columns]

In [104]:
result = calculate_product_monthly_prices(df_joined)
result

,Date,Product ID,Price
15636,2020-05-19,1,50.29
15639,2020-06-26,1,55.79
15640,2020-07-03,1,54.95
15709,2020-08-03,1,54.34
11181,2020-05-01,2,75.30
11183,2020-06-08,2,78.86
11187,2020-07-03,2,82.44
11255,2020-08-03,2,84.71
1610,2020-05-01,4,34.04
1607,2020-06-09,4,33.54


In [105]:
# Trying to shape the data so I can plot instrument price over time to display any trends
import calendar
graph_result = result.copy(deep=True)
graph_result['Date'] = graph_result['Date'].dt.month.apply(lambda x: calendar.month_abbr[x])
graph_result = graph_result.pivot(index='Date', columns='Product ID', values='Price')
graph_result.reset_index()
graph_result

# trace0 = go.Scatter(
#      x = result['Date'],
#      y = result['Price'])

# data = [trace]
# py.iplot(data)

Product ID,1,2,4,6,7,9,10,11,14,15,...,273,274,276,278,280,281,282,284,286,288
Date,,,,,,,,,,,,,,,,,,,,,
Aug,54.34,84.71,36.60,NaN,200.11,30.25,37.32,487.95,14.22,36.95,...,42.57,54.34,46.58,23.72,NaN,95.90,60.55,47.96,53.19,34.83
Jul,54.95,82.44,35.53,24.37,188.32,31.86,39.53,481.27,13.00,38.20,...,44.84,58.30,46.96,25.44,71.81,95.68,57.10,60.46,49.02,43.60
Jun,55.79,78.86,33.54,24.46,176.44,32.74,40.24,478.77,13.16,37.86,...,43.14,53.12,49.82,24.00,75.02,93.11,62.53,61.95,47.65,51.83
May,50.29,75.30,34.04,NaN,169.84,30.85,40.00,481.62,13.58,39.76,...,46.46,52.43,49.75,25.55,NaN,91.21,64.43,64.72,46.09,57.89
